In [7]:
from pathlib import Path
import configparser
import calendar
import time
import json
from github import Github

def check_rate_limit(github: Github):
        """Check the rate limit of the Github API."""
        core_rate_limit = github.get_rate_limit().core
        if core_rate_limit.remaining <= 5:
            print("Rate limit reached...")
            reset_timestamp = calendar.timegm(core_rate_limit.reset.timetuple())
            # add 5 seconds to be sure the rate limit has been reset)
            sleep_time = reset_timestamp - calendar.timegm(time.gmtime()) + 5
            print(f"Sleeping for {sleep_time} seconds...")
            time.sleep(sleep_time)
            print("Resuming after sleep...")

Load a GitHub token from `env.ini`

In [8]:
config = configparser.ConfigParser()
env_file = Path("../env.ini")
if not env_file.exists():
    raise ValueError("The env.ini file does not exist.")
else:
    config.read(env_file)

gh_token = json.loads(config.get("GITHUB", "TOKEN_LIST"))[0]

Perform query on GitHub

In [15]:
github = Github(gh_token)
print(github.get_user().login)
query = 'iterative/setup-cml path:.github/workflows extension:yaml'
result = github.search_code(query)
print(f'Total number of search results: {result.totalCount}')
# for file in result:
#     print(file.name)

louieQ
Total number of search results: 499


In [16]:
workflows: dict = {}

for file in result:
    
    time.sleep(0.5)
    # check_rate_limit(github)
    
    # Add repository as key in dictionary and set value to empty list
    if file.repository not in workflows:
        workflows[file.repository.html_url] = []

    # Add workflow file to list of workflows for repository
    workflows[file.repository].append(file.name)

In [17]:
workflows

{Repository(full_name="Etheredge-Works/gloves"): ['training.yaml'],
 Repository(full_name="sevillal/example_cml2"): ['cml.yaml'],
 Repository(full_name="alexpotv/FishFinder"): ['model_training.yaml'],
 Repository(full_name="ankithamrao/MLOPS-PSET2"): ['actions.yaml'],
 Repository(full_name="daniEL2371/abtest-mlops"): ['cml.yaml'],
 Repository(full_name="unlimt/cml_experiment"): ['cml.yaml'],
 Repository(full_name="nch1997/trainwithAWSgpu"): ['cml_dvc.yaml'],
 Repository(full_name="pgbayona/cicd"): ['model_evaluation.yaml'],
 Repository(full_name="SameC137/abtest-mlops"): ['cml.yaml'],
 Repository(full_name="dexterarmy/linear-regression"): ['cml.yaml'],
 Repository(full_name="seenumd/cml"): ['cml.yaml'],
 Repository(full_name="patrycjalazna/ium_cml"): ['cml.yaml'],
 Repository(full_name="yoelturner99/cicd_mlops"): ['model_evaluation.yaml'],
 Repository(full_name="Adelionek/ium_444507"): ['cml.yaml'],
 Repository(full_name="nebasam/Pharmaceutical-Sales-Forecasting"): ['cml.yaml'],
 Repos